In [1]:
#SVM
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score

C:\Users\aketu\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
df=pd.read_csv("pandas_label.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
Unnamed: 0                     194 non-null int64
sentence                       194 non-null object
support_keyword                194 non-null int64
after_conclusion_keyword       194 non-null int64
previous_conclusion_keyword    194 non-null int64
support_sentence               194 non-null int64
dtypes: int64(5), object(1)
memory usage: 9.2+ KB
None


In [3]:
print(df["support_sentence"].value_counts())

0    138
1     56
Name: support_sentence, dtype: int64


In [4]:
# target/input split
y = df["support_sentence"]
X = df.drop(["Unnamed: 0", "sentence", "support_sentence"], axis=1)

In [5]:
# setting random state
rs = 10

# train/test split
X_mat = X.as_matrix()
x_train3, x_test3, y_train3, y_test3 = train_test_split(X_mat, y, test_size=0.2, stratify=y, random_state=rs)

smt=SMOTE(random_state=20);
X_train3, Y_train3=smt.fit_sample(x_train3, y_train3);

C:\Users\aketu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [6]:
print (pd.Series(Y_train3).value_counts())
print (pd.Series(y_train3).value_counts())
print (pd.Series(y_test3).value_counts())

1    110
0    110
dtype: int64
0    110
1     45
Name: support_sentence, dtype: int64
0    28
1    11
Name: support_sentence, dtype: int64


In [7]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
print("Before scaling\n-------------")
for i in range(3):
    col = X_train3[:,i]
    print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".
          format(i, min(col), max(col), np.mean(col), np.std(col)))

X_train3 = scaler.fit_transform(X_train3, Y_train3)

print("After scaling\n-------------")
for i in range(3):
    col = X_train3[:,i]
    print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".
          format(i, min(col), max(col), np.mean(col), np.std(col)))

x_test3 = scaler.transform(x_test3)

Before scaling
-------------
Variable #0: min 0, max 1, mean 0.25 and std dev 0.44
Variable #1: min 0, max 2, mean 0.09 and std dev 0.30
Variable #2: min 0, max 2, mean 0.09 and std dev 0.30
After scaling
-------------
Variable #0: min -0.5843487097907777, max 1.711306935815849, mean 0.00 and std dev 1.00
Variable #1: min -0.3001501125938318, max 6.303152364470468, mean -0.00 and std dev 1.00
Variable #2: min -0.30015011259383223, max 6.303152364470477, mean -0.00 and std dev 1.00


In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(SVC(), param_grid={"C":[0.01, 0.1, 1, 10, 20, 30], 
                                       "gamma": [5, 10, 15, 20, 30, 1, 0.1, 0.01]}, cv=10)
grid.fit(X_train3, Y_train3)
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 0.01, 'gamma': 5} with a score of 0.79


In [18]:
clf = svm.SVC(C=0.01, gamma=5)
#cv_average_score=cross_val_score(model, X_train, Y_train, cv=10)
clf.fit(X_train3, Y_train3) 
#kernel="linear"

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=5, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [19]:
print("Train accuracy:", clf.score(X_train3, Y_train3))
print("Test accuracy:", clf.score(x_test3, y_test3))

y_pred3 = clf.predict(x_test3)
print(classification_report(y_test3, y_pred3))

Train accuracy: 0.7863636363636364
Test accuracy: 0.8205128205128205
              precision    recall  f1-score   support

           0       0.89      0.86      0.87        28
           1       0.67      0.73      0.70        11

    accuracy                           0.82        39
   macro avg       0.78      0.79      0.78        39
weighted avg       0.83      0.82      0.82        39



In [14]:
print(y_test3.values)
print(y_pred3)

[1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0]
[1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 1 0]


In [15]:
#print(clf.coef_)